In [ ]:
import numpy as np
import matplotlib.pyplot as plt

**Node class:** Ta klasa reprezentuje węzły w drzewie decyzyjnym. Węzły mogą być węzłami wewnętrznymi (węzłami decyzyjnymi) lub liśćmi (węzłami końcowymi). Węzły wewnętrzne zawierają informacje o cechach i progach podziału, a także odwołania do swoich lewego i prawego dziecka. Liście przechowują przewidywaną wartość dla podzbioru danych.

In [ ]:
class Node():

    def __init__(self, feature=None, threshold=None, left=None, right=None, gain=None, value=None):

        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.gain = gain
        self.value = value

**Klasa DecisionTree:**

- **split_data:** Ta metoda dzieli zbiór danych na dwa podzbiory na podstawie określonej cechy i progu.

- **Gini:** Ta funkcja oblicza nieczystość Gini dla danego zestawu etykiet. Jest ona zdefiniowana wzorem:
$$G(S) = 1 - \sum_{i=1}^{n} p_i^2,$$

  gdzie $p_i$ to prawdopodobieństwo wystąpienia każdej etykiety klasy $i$ w zbiorze danych $S$.

- **Information gain:** Ta funkcja oblicza zysk informacyjny poprzez znalezienie różnicy między nieczystością Gini węzła rodzica a ważoną sumą nieczystości Gini jego węzłów potomnych. Używa wzoru:
  $$IG(S, A) = G(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} G(S_v),$$
  gdzie:
  - $S$ to zbiór danych,
  - $A$ to atrybut,
  - $S_v$ to podzbiory $S$ dla każdej wartości $v$ atrybutu $A$.




- **best_split:** Ta funkcja znajduje najlepszy podział dla zbioru danych, iterując przez wszystkie cechy i ich unikalne wartości w celu obliczenia zysku informacyjnego. Zwraca indeks cechy, próg i dwa rezultujące zbiory danych, które maksymalizują zysk informacyjny.

- **calculate_leaf_value:** Ta funkcja oblicza wartość dla węzła liścia. Znajduje najczęściej występującą etykietę w danym zbiorze etykiet i przypisuje ją jako wartość węzła liścia.

- **build_tree:** Ta funkcja rekurencyjnie buduje drzewo decyzyjne, znajdując najlepszy podział w każdym węźle na podstawie zysku informacyjnego. Przerywa rekursję, gdy spełniony zostanie którykolwiek z kryteriów: minimalna liczba próbek lub maksymalna głębokość. Zwraca korzeń drzewa decyzyjnego.

- **fit:** Ta funkcja dopasowuje drzewo decyzyjne do danych treningowych. Konstruuje zestaw danych poprzez połączenie cech i etykiet, a następnie buduje drzewo za pomocą funkcji 'build_tree'.

- **predict:** Ta funkcja przewiduje etykiety dla próbek danych wejściowych za pomocą wytrenowanego drzewa decyzyjnego. Iteruje przez każdą próbkę i dokonuje predykcji, przechodząc przez drzewo aż do osiągnięcia węzła liścia.

- **make_prediction:** Ta funkcja przewiduje etykietę dla pojedynczej próbki wejściowej, przechodząc przez drzewo decyzyjne aż do osiągnięcia węzła liścia. Zwraca etykietę przypisaną do węzła liścia.


In [ ]:
class DecisionTree():

    def __init__(self, min_samples=2, max_depth=None):

        self.min_samples = min_samples
        self.max_depth = max_depth

    def split_data(self, dataset, feature, threshold):

        left_dataset = []
        right_dataset = []

        for row in dataset:
            if row[feature] <= threshold:
                left_dataset.append(row)
            else:
                right_dataset.append(row)

        left_dataset = np.array(left_dataset)
        right_dataset = np.array(right_dataset)
        return left_dataset, right_dataset

    def gini(self, y):
        gini = 1
        labels = np.unique(y)
        for label in labels:
            label_examples = y[y == label]
            pl = len(label_examples) / len(y)
            gini -= pl**2
        return gini


    def information_gain(self, parent, left, right):
        information_gain = 0
        parent_gini = self.gini(parent)
        weight_left = len(left) / len(parent)
        weight_right = len(right) / len(parent)
        gini_left, gini_right = self.gini(left), self.gini(right)
        weighted_gini = weight_left * gini_left + weight_right * gini_right
        information_gain = parent_gini - weighted_gini
        return information_gain


    def best_split(self, dataset, num_samples, num_features):

        best_split = {'gain': -1, 'feature': None, 'threshold': None}

        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]

            if isinstance(feature_values[0], float):
                thresholds = np.percentile(feature_values, np.linspace(0, 100, 100))
            else:
                thresholds = np.unique(feature_values)

            for threshold in thresholds:
                left_dataset, right_dataset = self.split_data(dataset, feature_index, threshold)
                if len(left_dataset) and len(right_dataset):
                    y, left_y, right_y = dataset[:, -1], left_dataset[:, -1], right_dataset[:, -1]
                    information_gain = self.information_gain(y, left_y, right_y)
                    if information_gain > best_split["gain"]:
                        best_split["feature"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_dataset"] = left_dataset
                        best_split["right_dataset"] = right_dataset
                        best_split["gain"] = information_gain
        return best_split


    def calculate_leaf_value(self, y):

        y = list(y)
        most_occuring_value = max(y, key=y.count)
        return most_occuring_value

    def build_tree(self, dataset, current_depth=0):

        X, y = dataset[:, :-1], dataset[:, -1]
        n_samples, n_features = X.shape

        if n_samples >= self.min_samples and (self.max_depth is None or current_depth < self.max_depth):
            best_split = self.best_split(dataset, n_samples, n_features)
            if best_split["gain"] > 0:
                left_dataset = best_split.get("left_dataset", None)
                right_dataset = best_split.get("right_dataset", None)

                if left_dataset is not None and right_dataset is not None:
                    left_node = self.build_tree(left_dataset, current_depth + 1)
                    right_node = self.build_tree(right_dataset, current_depth + 1)
                    return Node(best_split["feature"], best_split["threshold"],
                                left_node, right_node, best_split["gain"])

        leaf_value = self.calculate_leaf_value(y)
        return Node(value=leaf_value)

    def fit(self, X, y):

        dataset = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):

        predictions = []
        for x in X:
            prediction = self.make_prediction(x, self.root)
            predictions.append(prediction)
        np.array(predictions)
        return predictions

    def make_prediction(self, x, node):

        if node.value != None:
            return node.value
        else:
            feature = x[node.feature]
            if feature <= node.threshold:
                return self.make_prediction(x, node.left)
            else:
                return self.make_prediction(x, node.right)

    def predict_proba(self, X):
        predictions = []
        for x in X:
            prediction = self.make_prediction(x, self.root)
            proba = [1 - prediction, prediction]
            predictions.append(proba)
        return np.array(predictions)

In [ ]:
def tree_plot(node, depth=0, xmin=-2, xmax=2):
    if node is None:
        return

    if node.feature is not None:
        x_center = (xmin + xmax) / 2
        y_center = depth

        x_left = xmin if node.left is None else (xmin + x_center) / 2
        x_right = xmax if node.right is None else (xmax + x_center) / 2

        offset = 0.3
        threshold_rounded = round(node.threshold, 3)
        plt.text(x_center, y_center, f"Feature {node.feature} \n <=  {threshold_rounded}", ha='center', va='center',
                 bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2'))

        if node.left is not None:
            plt.arrow(x_center, y_center, x_left - x_center, -0.8, head_width=0.1, head_length=0.1, fc='black', ec='black')
            plt.text(x_left, y_center - 1, f"class:\n{node.left.value}", ha='center', va='center')
            tree_plot(node.left, depth - 1, xmin, x_center)

        if node.right is not None:
            plt.arrow(x_center, y_center, x_right - x_center, -0.8, head_width=0.1, head_length=0.1, fc='black', ec='black')
            plt.text(x_right, y_center - 1, f"class:\n{node.right.value}", ha='center', va='center')
            tree_plot(node.right, depth - 1, x_center, xmax)